In [1]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:


# Load the game schedule from the Excel file
file_path = 'game schedule.xlsx'
game_schedule = pd.read_excel(file_path)


game_schedule['DateTime'] = pd.to_datetime(game_schedule['Year'].astype(str) + '-' +
                                                   game_schedule['Month'] + '-' +
                                                   game_schedule['Date'].astype(str) + ' ' +
                                                   game_schedule['time'],
                                                   errors='coerce', format='%Y-%B-%d %H:%M')
print(game_schedule)

# Initialize the period for labeling
period_start = datetime(2011, 5, 1)
period_end = datetime(2011, 9, 30, 23, 59, 59)
current_interval = period_start
interval_duration = timedelta(seconds=20)

# Initialize a dictionary to hold the interval labels
interval_labels = {}

# Iterate through each interval in the specified period
while current_interval <= period_end:
    interval_labels[current_interval] = 0  # Default label is 0
    for _, row in game_schedule.iterrows():
        game_start = row['DateTime']
        pre_game_start = game_start - timedelta(hours=3)
        game_end = game_start + timedelta(hours=3)
        if pre_game_start <= current_interval < game_end:
            interval_labels[current_interval] = 1  # Set label to 1 for pre-game or during game
            break
    current_interval += interval_duration

# Optionally, convert the dictionary to a list for easier handling or exporting
interval_labels_list = list(interval_labels.items())

# To display or export the results, you can work with `interval_labels` or `interval_labels_list`
# For example, to print the first 10 labels:
print(interval_labels_list[:10])

    Year      Month  Date    time            DateTime
0   2011        May     3   19:10 2011-05-03 19:10:00
1   2011        May     4   19:10 2011-05-04 19:10:00
2   2011        May     5   19:10 2011-05-05 19:10:00
3   2011        May     6   19:10 2011-05-06 19:10:00
4   2011        May     7   18:10 2011-05-07 18:10:00
..   ...        ...   ...     ...                 ...
66  2011  September    17   13:10 2011-09-17 13:10:00
67  2011  September    18   13:10 2011-09-18 13:10:00
68  2011  September    26   19:10 2011-09-26 19:10:00
69  2011  September    27   19:10 2011-09-27 19:10:00
70  2011  September    28   19:10 2011-09-28 19:10:00

[71 rows x 5 columns]


In [ ]:

series_list = []
for i in range(len(interval_labels_list)):
    row=[interval_labels_list[i][0],interval_labels_list[i][1]]
    series_list.append(pd.Series(row, index=['datatime', 'label']))
    
labeldf = pd.DataFrame(series_list)
    
print(labeldf.head())    

In [ ]:
len(labeldf)

In [ ]:
import pandas as pd
labeldf.to_csv('gamelabel.csv', index=False) 

In [ ]:

gamelabel=pd.read_csv('gamelabel.csv')

#-----------------------------------------------------
new_datetime = gamelabel[['datatime']].copy()

new_datetime.loc[:, 'datetime'] = pd.to_datetime(new_datetime['datatime'])

new_datetime['weekday'] = new_datetime['datetime'].dt.dayofweek  # Monday=0, Sunday=6
new_datetime['hour'] = new_datetime['datetime'].dt.hour

#----------------Afternoon peak hour-----------------
APH_ind_5=[1,8,9,10,11,12,18,19,22,23,24,25]

APH_ind_6=[0,5,6,7,8,9,15,19,20,21,22,28]
APH_ind_61=[x+31 for x in APH_ind_6]

APH_ind_7=[4,5,6,7,10,11,12,17,18,19,20,21,24,25,26,27,30]
APH_ind_71=[x+31+30 for x in APH_ind_7]

APH_ind_8=[2,3,4,7,8,9,10,17,18,21,22,23,24]
APH_ind_81=[x+31+30+31 for x in APH_ind_8]

APH_ind_9=[1,5,6,14,18,19,20,21,22,28,29]
APH_ind_91=[x+31+30+31+31 for x in APH_ind_9]

APH_ind=APH_ind_5+APH_ind_61+APH_ind_71+APH_ind_81+APH_ind_91

for d in APH_ind:
    gamelabel.iloc[4320*d+15*180:4320*d+19*180,1]=2

#------------weekend peakhour-----------------
gamelabel.loc[(new_datetime['weekday']>=5) & (new_datetime['hour']>=12) & (new_datetime['hour']<=15) & (gamelabel['label'] == 0), 'label'] = 2  # free flow
#--------------Free flow-------------------    
gamelabel.loc[(new_datetime['hour']>=19) & (gamelabel['label'] == 0), 'label'] = 3  # free flow
gamelabel.loc[(new_datetime['weekday']>=5) & (gamelabel['label'] == 0), 'label'] = 3  # free flow
#--------------------------------------------
gamelabel.to_csv("gamelabel_1016_new.csv", index=False)



In [ ]:
#---plot---------------------------------------
plt.plot(range(4320), gamelabel['label'][4320:4320*2], marker='.', markersize=0.5)
#plt.xlim(0,4320)
plt.ylim(0,4.5)
plt.xticks(ticks=[1800,2160,2520,2880,3240,3600,3780], labels=['10h','12h','14h','16h','18h','20h','21h'])

plt.show()

plt.plot(range(4320), gamelabel['label'][4320*2:4320*3], marker='.', markersize=0.5)
#plt.xlim(0,4320)
plt.ylim(0,4.5)
plt.xticks(ticks=[1800,2160,2520,2880,3240,3600,3780], labels=['10h','12h','14h','16h','18h','20h','21h'])

plt.show()

plt.plot(range(4320), gamelabel['label'][4320*3:4320*4], marker='.', markersize=0.5)
#plt.xlim(0,4320)
plt.ylim(0,4.5)
plt.xticks(ticks=[1800,2160,2520,2880,3240,3600,3780], labels=['10h','12h','14h','16h','18h','20h','21h'])

plt.show()

plt.plot(range(4320), gamelabel['label'][4320*4:4320*5], marker='.', markersize=0.5)
#plt.xlim(0,4320)
plt.ylim(0,4.5)
plt.xticks(ticks=[1800,2160,2520,2880,3240,3600,3780], labels=['10h','12h','14h','16h','18h','20h','21h'])

plt.show()

plt.plot(range(4320), gamelabel['label'][4320*5:4320*6], marker='.', markersize=0.5)
#plt.xlim(0,4320)
plt.ylim(0,4.5)
plt.xticks(ticks=[1800,2160,2520,2880,3240,3600,3780], labels=['10h','12h','14h','16h','18h','20h','21h'])

plt.show()

plt.plot(range(4320), gamelabel['label'][4320*6:4320*7], marker='.', markersize=0.5)
#plt.xlim(0,4320)
plt.ylim(0,4.5)
plt.xticks(ticks=[1800,2160,2520,2880,3240,3600,3780], labels=['10h','12h','14h','16h','18h','20h','21h'])

plt.show()


label_counts = gamelabel['label'].value_counts()
